# Data Visualization and Analysis Notebook
<p>

In these Notebook, we will use Folium heatmaps to visualize the Risk Map and the Rainfall&Level Maps.

---
</p>

<br>

### Content 📖
**1. Risk Map of the Predicted Data**  
**2. Rainfall and Level Maps**  
&nbsp;&nbsp;&nbsp;&nbsp;**2.1** For Wet Day  
&nbsp;&nbsp;&nbsp;&nbsp;**2.2** For Typical Day  
  
**Operations:** Data unification, retrieval and visualization.

## 1. Flood Risk Map of the Predicted Data

The following heatmap illustrates the risk situation across the data coverage area, based on data from this project’s prediction module. 
- Zooming in and out on the map reveals more macro or detailed information. This is an **interactive and user-friendly** information map.
- On the map, locations with the **highest risk** (risk: 7) **are marked** to alert users to areas with the greatest potential for incidents. 
- Clicking on **these markers provides more detailed information** about the location, including postcode, household, and headcount (if these details are available in the resource datasets).

In [ ]:
import folium
import pandas as pd
from folium.plugins import HeatMap
import branca.colormap as cm

def Risk_HeatMap(file_path):
    df = pd.read_csv(file_path)
    df['riskLabel'] = pd.to_numeric(df['riskLabel'], errors='coerce')

    m = folium.Map(location=[51.5074, -1], zoom_start=10)

    # Add a heat map layer
    heat_data = [[row['Latitude'], row['Longitude'], row['riskLabel']] for _, row in df.iterrows()]
    HeatMap(heat_data, radius=15, blur=10, max_zoom=1).add_to(m)

    # Create a color map for the color bar
    colormap = cm.LinearColormap(colors=['blue', 'green', 'yellow', 'red'], 
                                vmin=min(df['riskLabel']), 
                                vmax=max(df['riskLabel']),
                                caption='Risk Level')
    colormap.add_to(m)

    for _, row in df.iterrows():
        if row["riskLabel"] == 7:
            households, headcount = get_house_head(row["postcode"])
            if households.empty or headcount.empty:
                folium.Marker(
                    location=[row['Latitude'], row['Longitude']],
                    popup=f"Postcode: {row['postcode']}",
                    icon=folium.Icon(color='red', icon='info-sign')).add_to(m)
            else:
                folium.Marker(
                    location=[row['Latitude'], row['Longitude']],
                    popup=f"Postcode: {row['postcode']}, Households: {households.values}, Headcount: {headcount.values}",
                    icon=folium.Icon(color='red', icon='info-sign')).add_to(m)

    # Add a custom legend to the map
    legend_html = """
    <div style="
        position: fixed;
        bottom: 20px; left: 20px; width: 200px; height: 90px;
        background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
        padding: 10px;">
        <strong>Info-sign Marker:</strong><br>
        Most Dangerous Area (Risk:7)
    </div>
    """

    m.get_root().html.add_child(folium.Element(legend_html))

    return m

def get_house_head(postcode):
    df = pd.read_csv("./resources_data/sector_data.csv")

    postcode_first = postcode.split()[0]  # first part
    postcode_second = postcode.split()[1][0]  # first character of the second part
    postcodeSector = postcode_first + " " + postcode_second

    df = df[df["postcodeSector"] == postcodeSector]
    return df[["households"]], df[["headcount"]]

path = "./output/postcodes_unlabelled_predicted.csv"
map = Risk_HeatMap(path)

map

# 2. Rainfall and Level Maps ☔️ 🌊
Two separate maps are created here to display wet and typical day conditions. Each map is composed of two heatmaps, representing rainfall and water level. By inputting different station names into the function, different maps are generated in real-time, enabling offline interactive operation.

**For any map, users can choose to display only one of the heatmaps—either rainfall or water level**—allowing them to focus more closely on the specific water level values. Alternatively, both heatmaps can be overlaid to observe the combined situation.

The `station_name` is selectable, and users can choose the desired station through interaction with the webpage. This effect is achieved by combining station data with the rainfall and water level datasets. In the following example, we use the `"Rainfall station"` as a sample input.

The units of the data were inconsistent, which We suspected was due to differing measurement methods. Therefore, **we standardized all units to "meters" or their equivalent**.

## 2.1 For Wet Day 🌧️
Data from `wet_day.csv`:

In [ ]:
def normalise_rainfall(df):
    """
    Normalize rainfall values to millimeters (mm) if the unit is not already in mm.

    If the `unitName` is not "mm", the corresponding `value` is multiplied by 1000 
    to convert it to millimeters, and the `unitName` is updated to "mm".
    """
    df.loc[df["unitName"] != "mm", "value"] *= 1000
    df.loc[df["unitName"] != "mm", "unitName"] = "mm"
    return df

def Rainfall_merged_data(station_name):
    """ 
    Merge station data with typical and wet day rainfall data for the specified station.

    This function reads the station, typical day, and wet day rainfall data, filters 
    for the given station, normalizes the rainfall values, and merges them into a single DataFrame.
    """
    # read the station data
    station_df = pd.read_csv("./resources_data/stations.csv")
    station_info = station_df[station_df["stationName"] == station_name][['stationReference', 'latitude', 'longitude']]
    
    if station_info.empty:
        print(f"No station found with name: {station_name}")
        return None
    
    # read the rainfall data
    typical_data = pd.read_csv("./resources_data/typical_day.csv")
    wet_data = pd.read_csv("./resources_data/wet_day.csv")
    typical_data['value'] = pd.to_numeric(typical_data['value'], errors='coerce')
    wet_data['value'] = pd.to_numeric(wet_data['value'], errors='coerce')

    # normalise the rainfall data
    typical_data = normalise_rainfall(typical_data)
    wet_data = normalise_rainfall(wet_data)

    # merge the station data with the rainfall data
    merged_data = pd.merge(
        station_info,
        typical_data[["stationReference", "parameter", "value"]],
        on="stationReference",
        how="left"
    ).rename(columns={"parameter": "typical_parameter", "value": "typical_value"})

    merged_data = pd.merge(
        merged_data,
        wet_data[["stationReference", "parameter", "value"]],
        on="stationReference",
        how="left"
    ).rename(columns={"parameter": "wet_parameter", "value": "wet_value"})

    merged_data.drop_duplicates(inplace=True)
    merged_data.dropna(inplace=True)

    return merged_data

def Wet_Rainfall_HeatMap(station_name):
    """
    Create a heatmap with rainfall and water level data for wet days based on the given station.

    This function generates a folium map with two heatmaps: one for rainfall and one for water levels,
    using data from the specified station.
    """
    merged_data = Rainfall_merged_data(station_name)

    m = folium.Map(location=[merged_data['latitude'][0], merged_data['longitude'][0]], zoom_start=7)

    # Filter data for rainfall
    rainfall_data = [
        [row['latitude'], row['longitude'], row['wet_value']]
        for _, row in merged_data.iterrows() if row['wet_parameter'] == 'rainfall'
    ]

    # Filter data for level
    level_data = [
        [row['latitude'], row['longitude'], row['wet_value']]
        for _, row in merged_data.iterrows() if row['wet_parameter'] == 'level'
    ]

    # Create color maps for the color bars
    merged_vmin = 0.2
    merged_vmax = 0.8
    rainfall_colormap = cm.LinearColormap(['blue', 'yellow', 'red'], vmin=merged_vmin, vmax=merged_vmax, caption='Rainfall (mm)')
    level_colormap = cm.LinearColormap(['green', 'orange', 'purple'], vmin=merged_vmin, vmax=merged_vmax, caption='Water Level (mm)')
    rainfall_colormap.add_to(m)
    level_colormap.add_to(m)
    
    # Add HeatMap for rainfall with a gradient (e.g., blue to red)
    heatmap_rainfall = folium.FeatureGroup(name='Rainfall Heatmap')
    HeatMap(
        rainfall_data,
        radius=15,
        blur=10,
        max_zoom=1,
        gradient={0.2: 'blue', 0.5: 'yellow', 0.8: 'red'}
    ).add_to(heatmap_rainfall)
    heatmap_rainfall.add_to(m)

    # Add HeatMap for level with a different gradient (e.g., green to purple)
    heatmap_level = folium.FeatureGroup(name='Level Heatmap')
    HeatMap(
        level_data,
        radius=15,
        blur=10,
        max_zoom=1,
        gradient={0.2: 'green', 0.5: 'orange', 0.8: 'purple'}
    ).add_to(heatmap_level)
    heatmap_level.add_to(m)

    # Add layer control to toggle between heatmaps
    folium.LayerControl().add_to(m)

    return m

station_name = "Rainfall station"
map = Wet_Rainfall_HeatMap(station_name)

map

## 2.2 For Typical Day ☁️
Data from `typical_day.csv`:

In [ ]:
def Typical_Rainfall_HeatMap(station_name):
    """
    Create a heatmap with rainfall and water level data for a typical day based on the given station.

    This function generates a folium map with two heatmaps: one for rainfall and one for water levels,
    using data from the specified station.
    """
    merged_data = Rainfall_merged_data(station_name)

    m = folium.Map(location=[merged_data['latitude'][0], merged_data['longitude'][0]], zoom_start=7)

    # Filter data for rainfall
    rainfall_data = [
        [row['latitude'], row['longitude'], row['typical_value']]
        for _, row in merged_data.iterrows() if row['typical_parameter'] == 'rainfall'
    ]

    # Filter data for level
    level_data = [
        [row['latitude'], row['longitude'], row['typical_value']]
        for _, row in merged_data.iterrows() if row['typical_parameter'] == 'level'
    ]

    # Create color maps for the color bars
    merged_vmin = 0.2
    merged_vmax = 0.8
    rainfall_colormap = cm.LinearColormap(['blue', 'yellow', 'red'], vmin=merged_vmin, vmax=merged_vmax, caption='Rainfall (mm)')
    level_colormap = cm.LinearColormap(['green', 'orange', 'purple'], vmin=merged_vmin, vmax=merged_vmax, caption='Water Level (mm)')
    rainfall_colormap.add_to(m)
    level_colormap.add_to(m)
    
    # Add HeatMap for rainfall with a gradient (e.g., blue to red)
    heatmap_rainfall = folium.FeatureGroup(name='Rainfall Heatmap')
    HeatMap(
        rainfall_data,
        radius=15,
        blur=10,
        max_zoom=1,
        gradient={0.2: 'blue', 0.5: 'yellow', 0.8: 'red'}
    ).add_to(heatmap_rainfall)
    heatmap_rainfall.add_to(m)

    # Add HeatMap for level with a different gradient (e.g., green to purple)
    heatmap_level = folium.FeatureGroup(name='Level Heatmap')
    HeatMap(
        level_data,
        radius=15,
        blur=10,
        max_zoom=1,
        gradient={0.2: 'green', 0.5: 'orange', 0.8: 'purple'}
    ).add_to(heatmap_level)
    heatmap_level.add_to(m)

    # Add layer control to toggle between heatmaps
    folium.LayerControl().add_to(m)

    return m

station_name = "Rainfall station"
map = Typical_Rainfall_HeatMap(station_name)

map

It is evident that on wet days, rainfall and water level values across the UK decrease significantly, and high-value risk areas also become less frequent.